In [45]:
#from google.colab import drive
#drive.mount('/content/drive')
#!cp "/content/drive/My Drive/198-adversarial-ml/Kaggle-Fake-News/train.csv" "train.csv"

In [46]:
import pandas as pd
import itertools
import string
import re

In [47]:
df = pd.read_csv('train.csv')

In [48]:
adv_list = ['almost', 'badly', 'fully', 'extremely', 'hardly', 'very', 'fairly', 'really', 'much', 'perfectly',
            'well', 'nearly', 'less', 'indeed', 'too', 'so', 'quite', 'rather', 'highly', 'totally']
special = ['!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~', '“', '”', '‘', '’']

Adds special characters before and after the adverb.

In [49]:
df = df[df['text'].notnull()]
adv_list_sp = []
adv_list_sp.append(adv_list)
for i in adv_list:
  temp_list = []
  for j in special:
    temp_list.append(i + j)
    temp_list.append(j + i)
  adv_list_sp.append(temp_list)

adv_list = [item for sublist in adv_list_sp for item in sublist]
print(adv_list)

['almost', 'badly', 'fully', 'extremely', 'hardly', 'very', 'fairly', 'really', 'much', 'perfectly', 'well', 'nearly', 'less', 'indeed', 'too', 'so', 'quite', 'rather', 'highly', 'totally', 'almost!', '!almost', 'almost"', '"almost', 'almost#', '#almost', 'almost$', '$almost', 'almost%', '%almost', 'almost&', '&almost', "almost'", "'almost", 'almost(', '(almost', 'almost)', ')almost', 'almost*', '*almost', 'almost+', '+almost', 'almost,', ',almost', 'almost-', '-almost', 'almost.', '.almost', 'almost/', '/almost', 'almost:', ':almost', 'almost;', ';almost', 'almost<', '<almost', 'almost=', '=almost', 'almost>', '>almost', 'almost?', '?almost', 'almost@', '@almost', 'almost[', '[almost', 'almost\\', '\\almost', 'almost]', ']almost', 'almost^', '^almost', 'almost_', '_almost', 'almost`', '`almost', 'almost{', '{almost', 'almost|', '|almost', 'almost}', '}almost', 'almost~', '~almost', 'almost“', '“almost', 'almost”', '”almost', 'almost‘', '‘almost', 'almost’', '’almost', 'badly!', '!badl

Testing of modifier

In [50]:
df_test = pd.DataFrame(columns=["label", "text"], data=[[0, 'really "really ,really something']])


In [51]:
df_test['text_new'] = df_test['text'].apply(lambda s: ' '.join([w for w in s.split() if w.lower() not in adv_list]))

display(df_test)

,label,text,text_new
0,0,"really ""really ,really something",something


Creating modified dataset

In [52]:
#attribution: https://github.com/ljyflores/fake-news-adversarial-benchmark/blob/master/polarity_preprocessing.ipynb
df['text_new'] = df['text'].apply(lambda s: ' '.join([w for w in s.split() if w.lower() not in adv_list]))

df_old = df[['id','title','author','text','label']]
df_new  = df[['id','title','author','text_new','label']].rename(columns={'text_new':'text'})

In [54]:
#df_old.to_csv('/content/drive/My Drive/198-adversarial-ml/Kaggle-Fake-News/train_old.csv', index=False)
#df_new.to_csv('/content/drive/My Drive/198-adversarial-ml/Kaggle-Fake-News/train_new.csv', index=False)

In [55]:
#!cp "/content/drive/My Drive/198-adversarial-ml/Kaggle-Fake-News/train_old.csv" "train_old.csv"
#!cp "/content/drive/My Drive/198-adversarial-ml/Kaggle-Fake-News/train_new.csv" "train_new.csv"

Validation of degree adverbs removal

In [56]:
#attribution: https://stackoverflow.com/questions/60873474/find-specific-words-on-dataframe
ids_old = df_old.text.str.contains('really$|really-|really ', flags = re.IGNORECASE, regex = True, na = False)
ids_new = df_new.text.str.contains('really$|really-|really ', flags = re.IGNORECASE, regex = True, na = False)

3937 rows with adverb "really".

In [57]:
df_old[ids_old]

,id,title,author,text,label
9,9,"A Back-Channel Plan for Ukraine and Russia, Co...",Megan Twohey and Scott Shane,A week before Michael T. Flynn resigned as nat...,0
11,11,"BBC Comedy Sketch ""Real Housewives of ISIS"" Ca...",Chris Tomlinson,The BBC produced spoof on the “Real Housewives...,0
14,14,"Re: Yes, There Are Paid Government Trolls On S...",AnotherAnnie,"Yes, There Are Paid Government Trolls On Socia...",1
15,15,"In Major League Soccer, Argentines Find a Home...",Jack Williams,Guillermo Barros Schelotto was not the first A...,0
21,21,"Monica Lewinsky, Clinton Sex Scandal Set for ’...",Jerome Hudson,"Screenwriter Ryan Murphy, who has produced the...",0
...,...,...,...,...,...
20758,20758,Trump’s Opponents See Normal Americans as Depl...,pcr3,Trump’s Opponents See Normal Americans as Depl...,1
20765,20765,NFL Preview: Championship Match-Ups Prove Team...,Daniel Leberfeld,"The NFL is a league, so it should come as no...",0
20773,20773,Australia to hunt down anti-vax nurses and pro...,Vicki Batts,Australia to hunt down anti-vax nurses and pro...,1
20784,20784,Comment on World Heaves Sigh of Relief after T...,Debbie Menon,Finian Cunningham has written extensively on...,1


Only 7 rows with the adverb "really" are left in the modified dataframe. Double consecutive instances of punctuations and other special characters have not been added to the list of words to be removed.

In [59]:
df_new[ids_new]

,id,title,author,text,label
2610,2610,The Mainstream Media Monopoly Is Over,No Author,"The arrival of new media, like The Duran and m...",1
4879,4879,Food Synergy,Michael Greger M.D. FACLM,Blair Rollin: Nice quote from The China Study....,1
7203,7203,Comment on 2:00PM Water Cooler 10/28/2016 by J...,JohnnyGL,"“Political Airpower, Part I: Say No to the No-...",1
9066,9066,An open letter to Fareed Zakaria from The Peop...,Pamalinsky,"1/18/2016, 12:59 am Fareed Zakaria can dish it...",1
11960,11960,“The Radical Left Seems Ready To Go To War” As...,Contributing Author,"Read by 2,559 people This article was written ...",1
15973,15973,Jonah Hill Is No Joke - The New York Times,Molly Young,You can tell a lot about a person by watching ...,0
18440,18440,President-Elect Donald Trump's First Televised...,informationclearinghouse.info,President-Elect Donald Trump's First Televised...,1
